In [1]:
pip install boto3

Note: you may need to restart the kernel to use updated packages.


In [2]:
import boto3

In [3]:
import time 

In [4]:
import pandas as pd

In [7]:
from io import StringIO

In [8]:
AWS_ACCESS_KEY=""
AWS_SECRET_KEY=""
AWS_REGION="ap-south-1"
SCHEMA_NAME="covid_19"
S3_STAGING_DIR="s3://output-sumer1/output/"
S3_BUCKET_NAME="output-sumer1"
S3_OUTPUT_DIRECTORY="output"

In [9]:
athena_client=boto3.client("athena",aws_access_key_id =AWS_ACCESS_KEY,
                          aws_secret_access_key=AWS_SECRET_KEY,
                          region_name=AWS_REGION,)

In [10]:
Dict={}
def download_and_load_query_results(
    client:boto3.client,query_response:Dict
) -> pd.DataFrame:
    while True:
        try:
            #This function only loads the first 1000 rows
            client.get_query_results(
                QueryExecutionId=query_response["QueryExecutionId"]
            )
            break
        except Exception as err:
            if "not yet finished" in str(err):
                time.sleep(0.001)
            else:
                raise err
    temp_file_location:str="athena_query_results.csv"
    s3_client=boto3.client(
        "s3",
        aws_access_key_id=AWS_ACCESS_KEY,
        aws_secret_access_key=AWS_SECRET_KEY,
        region_name=AWS_REGION,
    )
        
    s3_client.download_file(
        S3_BUCKET_NAME,
        f"{S3_OUTPUT_DIRECTORY}/{query_response['QueryExecutionId']}.csv",
        temp_file_location,
        )
        
    return pd.read_csv(temp_file_location) 

In [13]:
response = athena_client.start_query_execution(
    QueryString="SELECT * FROM enigma_jhud LIMIT",
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"}
    }
)

In [14]:
response

{'QueryExecutionId': '97af4525-ae25-4b48-a463-f8abffa49908',
 'ResponseMetadata': {'RequestId': '3b0d299c-28cc-43ab-93f2-878c20ab32ba',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 26 Feb 2024 22:23:41 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '59',
   'connection': 'keep-alive',
   'x-amzn-requestid': '3b0d299c-28cc-43ab-93f2-878c20ab32ba'},
  'RetryAttempts': 0}}

In [15]:
enigma_jhud=download_and_load_query_results(athena_client,response)

In [16]:
enigma_jhud.head()

,fips,admin2,province_state,country_region,last_update,latitude,longitude,confirmed,deaths,recovered,active,combined_key
0,NaN,NaN,Anhui,China,2020-01-22T17:00:00,31.826,117.226,1.0,NaN,NaN,NaN,"""Anhui"
1,NaN,NaN,Beijing,China,2020-01-22T17:00:00,40.182,116.414,14.0,NaN,NaN,NaN,"""Beijing"
2,NaN,NaN,Chongqing,China,2020-01-22T17:00:00,30.057,107.874,6.0,NaN,NaN,NaN,"""Chongqing"
3,NaN,NaN,Fujian,China,2020-01-22T17:00:00,26.079,117.987,1.0,NaN,NaN,NaN,"""Fujian"
4,NaN,NaN,Gansu,China,2020-01-22T17:00:00,36.061,103.834,NaN,NaN,NaN,NaN,"""Gansu"


In [37]:
response = athena_client.start_query_execution(
    QueryString="SELECT * FROM enigma_nytimes_data_in_usa_us_county",
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"}
    }
)

enigma_nytimes_data_in_usa_us_county=download_and_load_query_results(athena_client,response)

enigma_nytimes_data_in_usa_us_county.head()

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0
1,2020-01-22,Snohomish,Washington,53061.0,1,0
2,2020-01-23,Snohomish,Washington,53061.0,1,0
3,2020-01-24,Cook,Illinois,17031.0,1,0
4,2020-01-24,Snohomish,Washington,53061.0,1,0


In [38]:
response = athena_client.start_query_execution(
    QueryString="SELECT * FROM enigma_nytimes_data_in_usa_us_states",
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"}
    }
)

enigma_nytimes_data_in_usa_us_states=download_and_load_query_results(athena_client,response)

enigma_nytimes_data_in_usa_us_states.head()

,date,state,fips,cases,deaths
0,2020-01-21,Washington,53,1,0
1,2020-01-22,Washington,53,1,0
2,2020-01-23,Washington,53,1,0
3,2020-01-24,Illinois,17,1,0
4,2020-01-24,Washington,53,1,0


In [39]:
response = athena_client.start_query_execution(
    QueryString="SELECT * FROM rearc_covid_19_testing_data_states_daily",
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"}
    }
)

rearc_covid_19_testing_data_states_daily=download_and_load_query_results(athena_client,response)

rearc_covid_19_testing_data_states_daily.head()

,date,state,positive,probablecases,negative,pending,totaltestresultssource,totaltestresults,hospitalizedcurrently,hospitalizedcumulative,...,dataqualitygrade,deathincrease,hospitalizedincrease,hash,commercialscore,negativeregularscore,negativescore,positivescore,score,grade
0,20210307,AK,56886.0,NaN,NaN,NaN,totalTestsViral,1731628.0,33.0,1293.0,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
1,20210307,AL,499819.0,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788.0,494.0,45976.0,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2,20210307,AR,324818.0,69092.0,2480716.0,NaN,totalTestsViral,2736442.0,335.0,14926.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
3,20210307,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,...,NaN,0,0,96d23f888c995b9a7f3b4b864de6414f45c728ff,0,0,0,0,0,NaN
4,20210307,AZ,826454.0,56519.0,3073010.0,NaN,totalTestsViral,7908105.0,963.0,57907.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [40]:
response = athena_client.start_query_execution(
    QueryString="SELECT * FROM rearc_covid_19_testing_data_us",
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"}
    }
)

rearc_covid_19_testing_data_us_total_latest=download_and_load_query_results(athena_client,response)

rearc_covid_19_testing_data_us_total_latest.head()

,positive,negative,pending,hospitalizedcurrently,hospitalizedcumulative,inicucurrently,inicucumulative,onventilatorcurrently,onventilatorcumulative,recovered,hash,lastmodified,death,hospitalized,total,totaltestresults,posneg,notes
0,1061101,5170081,2775,53793,111955,9486,4192,4712,373,153947,95064ba29ccbc20dbec397033dfe4b1f45137c99,2020-05-01T09:12:31.891Z,57266,111955,6233957,6231182,6231182,"""NOTE: """"total"""""


In [41]:
response = athena_client.start_query_execution(
    QueryString="SELECT * FROM rearc_covid_19_testing_data_us_daily",
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"}
    }
)

rearc_covid_19_testing_data_us_daily=download_and_load_query_results(athena_client,response)

rearc_covid_19_testing_data_us_daily.head()

,date,states,positive,negative,pending,hospitalizedcurrently,hospitalizedcumulative,inicucurrently,inicucumulative,onventilatorcurrently,...,lastmodified,recovered,total,posneg,deathincrease,hospitalizedincrease,negativeincrease,positiveincrease,totaltestresultsincrease,hash
0,20210307,56,28755524.0,74579770.0,11808.0,40212.0,878613.0,8137.0,45475.0,2801.0,...,2021-03-07T24:00:00Z,NaN,0,0,839,726,130414,41265,1156241,8b26839690cd05c0cef69cb9ed85641a76b5e78e
1,20210306,56,28714259.0,74449356.0,11783.0,41401.0,877887.0,8409.0,45453.0,2811.0,...,2021-03-06T24:00:00Z,NaN,0,0,1674,503,142201,59620,1409138,d0c0482ea549c9d5c04a7c86acb6fc6a8095a592
2,20210305,56,28654639.0,74307155.0,12213.0,42541.0,877384.0,8634.0,45373.0,2889.0,...,2021-03-05T24:00:00Z,NaN,0,0,2221,2781,271917,68787,1744417,a35ea4289cec4bb55c9f29ae04ec0fd5ac4e0222
3,20210304,56,28585852.0,74035238.0,12405.0,44172.0,874603.0,8970.0,45293.0,2973.0,...,2021-03-04T24:00:00Z,NaN,0,0,1743,1530,177957,65487,1590984,a19ad6379a653834cbda3093791ad2c3b9fab5ff
4,20210303,56,28520365.0,73857281.0,11778.0,45462.0,873073.0,9359.0,45214.0,3094.0,...,2021-03-03T24:00:00Z,NaN,0,0,2449,2172,267001,66836,1406795,9e1d2afda1b0ec243060d6f68a7134d011c0cb2a


In [42]:
response = athena_client.start_query_execution(
    QueryString="SELECT * FROM static_datasets_countrycodeqs",
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"}
    }
)

static_datasets_countrycode=download_and_load_query_results(athena_client,response)

static_datasets_countrycode.head()

,country,alpha-2 code,alpha-3 code,numeric code,latitude,longitude
0,Afghanistan,AF,AFG,4.0,33.0000,65.0
1,Albania,AL,ALB,8.0,41.0000,20.0
2,Algeria,DZ,DZA,12.0,28.0000,3.0
3,American Samoa,AS,ASM,16.0,-14.3333,-170.0
4,Andorra,AD,AND,20.0,42.5000,1.6


In [43]:
response = athena_client.start_query_execution(
    QueryString="SELECT * FROM static_datasets_county_population",
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"}
    }
)

static_datasets_CountyPopulation=download_and_load_query_results(athena_client,response)

static_datasets_CountyPopulation.head()

,id,id2,county,state,population estimate 2018
0,0500000US01001,1001,Autauga,Alabama,55601
1,0500000US01003,1003,Baldwin,Alabama,218022
2,0500000US01005,1005,Barbour,Alabama,24881
3,0500000US01007,1007,Bibb,Alabama,22400
4,0500000US01009,1009,Blount,Alabama,57840


In [44]:
response = athena_client.start_query_execution(
    QueryString="SELECT * FROM static_datasets_states_abv",
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"}
    }
)

static_datasets_state_abv=download_and_load_query_results(athena_client,response)

static_datasets_state_abv.head()

,col0,col1
0,State,Abbreviation
1,Alabama,AL
2,Alaska,AK
3,Arizona,AZ
4,Arkansas,AR


In [45]:
response = athena_client.start_query_execution(
    QueryString="SELECT * FROM rearc_usa_hospital_beds",
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"}
    }
)

rearc_usa_hospital_beds=download_and_load_query_results(athena_client,response)

rearc_usa_hospital_beds.head()

,objectid,hospital_name,hospital_type,hq_address,hq_address1,hq_city,hq_state,hq_zip_code,county_name,state_name,...,num_licensed_beds,num_staffed_beds,num_icu_beds,adult_icu_beds,pedi_icu_beds,bed_utilization,avg_ventilator_usage,potential_increase_in_bed_capac,latitude,longtitude
0,1,Phoenix VA Health Care System (AKA Carl T Hayd...,VA Hospital,650 E Indian School Rd,NaN,Phoenix,AZ,85012,Maricopa,Arizona,...,129.0,129.0,0,0,NaN,NaN,0.0,0,33.495498,-112.066157
1,2,Southern Arizona VA Health Care System,VA Hospital,3601 S 6th Ave,NaN,Tucson,AZ,85723,Pima,Arizona,...,295.0,295.0,2,2,NaN,NaN,2.0,0,32.181263,-110.965885
2,3,VA Central California Health Care System,VA Hospital,2615 E Clinton Ave,NaN,Fresno,CA,93703,Fresno,California,...,57.0,57.0,2,2,NaN,NaN,2.0,0,36.773324,-119.779742
3,4,VA Connecticut Healthcare System - West Haven ...,VA Hospital,950 Campbell Ave,NaN,West Haven,CT,6516,New Haven,Connecticut,...,216.0,216.0,1,1,NaN,NaN,2.0,0,41.284400,-72.957610
4,5,Wilmington VA Medical Center,VA Hospital,1601 Kirkwood Hwy,NaN,Wilmington,DE,19805,New Castle,Delaware,...,60.0,60.0,0,0,NaN,NaN,1.0,0,39.740206,-75.606532


In [46]:
new_header=static_datasets_state_abv.iloc[0]

In [47]:
new_header

col0           State
col1    Abbreviation
Name: 0, dtype: object

In [48]:
static_datasets_state_abv=static_datasets_state_abv[1:]

In [49]:
static_datasets_state_abv.columns=new_header

In [50]:
static_datasets_state_abv.head()

,State,Abbreviation
1,Alabama,AL
2,Alaska,AK
3,Arizona,AZ
4,Arkansas,AR
5,California,CA


In [105]:
factCovid_1=enigma_jhud[['fips','province_state','country_region','confirmed','deaths','recovered','active']]
factCovid_2=rearc_covid_19_testing_data_states_daily[['fips','date','positive','negative','hospitalizedcurrently','hospitalized','hospitalizeddischarged']]
factCovid=pd.merge(factCovid_1,factCovid_2, on='fips',how='inner')

In [ ]:
factCovid.head()

In [53]:
factCovid.shape

(26418, 13)

In [54]:
dimRegion_1=enigma_jhud[['fips','province_state','country_region','latitude','longitude']]
dimRegion_2=enigma_nytimes_data_in_usa_us_county[['fips','county','state']]
dimRegion=pd.merge(dimRegion_1,dimRegion_2, on='fips',how='inner')

In [55]:
dimRegion.head()

,fips,province_state,country_region,latitude,longitude,county,state
0,NaN,Anhui,China,31.826,117.226,New York City,New York
1,NaN,Anhui,China,31.826,117.226,Unknown,Rhode Island
2,NaN,Anhui,China,31.826,117.226,New York City,New York
3,NaN,Anhui,China,31.826,117.226,Unknown,Rhode Island
4,NaN,Anhui,China,31.826,117.226,New York City,New York


In [114]:
dimRegion=dimRegion.head(2000)

In [56]:
dimHospital=rearc_usa_hospital_beds[['fips','state_name','latitude','longtitude','hq_address','hospital_type','hospital_name','hq_city','hq_state']]

In [57]:
dimHospital.head()

,fips,state_name,latitude,longtitude,hq_address,hospital_type,hospital_name,hq_city,hq_state
0,4013.0,Arizona,33.495498,-112.066157,650 E Indian School Rd,VA Hospital,Phoenix VA Health Care System (AKA Carl T Hayd...,Phoenix,AZ
1,4019.0,Arizona,32.181263,-110.965885,3601 S 6th Ave,VA Hospital,Southern Arizona VA Health Care System,Tucson,AZ
2,6019.0,California,36.773324,-119.779742,2615 E Clinton Ave,VA Hospital,VA Central California Health Care System,Fresno,CA
3,9009.0,Connecticut,41.284400,-72.957610,950 Campbell Ave,VA Hospital,VA Connecticut Healthcare System - West Haven ...,West Haven,CT
4,10003.0,Delaware,39.740206,-75.606532,1601 Kirkwood Hwy,VA Hospital,Wilmington VA Medical Center,Wilmington,DE


In [58]:
dimDate=rearc_covid_19_testing_data_states_daily[['fips','date']]

In [59]:
dimDate.head()

,fips,date
0,2,20210307
1,1,20210307
2,5,20210307
3,60,20210307
4,4,20210307


In [60]:
dimDate['date']=pd.to_datetime(dimDate['date'],format='%Y%m%d')

C:\Users\sumer\AppData\Local\Temp\ipykernel_9792\4062609722.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dimDate['date']=pd.to_datetime(dimDate['date'],format='%Y%m%d')


In [61]:
dimDate.head()

,fips,date
0,2,2021-03-07
1,1,2021-03-07
2,5,2021-03-07
3,60,2021-03-07
4,4,2021-03-07


In [62]:
dimDate['year']=dimDate['date'].dt.year
dimDate['month']=dimDate['date'].dt.month
dimDate['day_of_week']=dimDate['date'].dt.dayofweek

C:\Users\sumer\AppData\Local\Temp\ipykernel_9792\2089747518.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dimDate['year']=dimDate['date'].dt.year
C:\Users\sumer\AppData\Local\Temp\ipykernel_9792\2089747518.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dimDate['month']=dimDate['date'].dt.month
C:\Users\sumer\AppData\Local\Temp\ipykernel_9792\2089747518.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value 

In [63]:
dimDate.head()

,fips,date,year,month,day_of_week
0,2,2021-03-07,2021,3,6
1,1,2021-03-07,2021,3,6
2,5,2021-03-07,2021,3,6
3,60,2021-03-07,2021,3,6
4,4,2021-03-07,2021,3,6


In [64]:
bucket='output-sumer1'

In [115]:
csv_buffer=StringIO()

In [116]:
csv_buffer

In [117]:
dimRegion.to_csv(csv_buffer)


In [118]:
s3_resource = boto3.resource('s3',
                                  aws_access_key_id=AWS_ACCESS_KEY,
                                  aws_secret_access_key= AWS_SECRET_KEY)

In [119]:
s3_resource=boto3.resource('s3',
                                  aws_access_key_id=AWS_ACCESS_KEY,
                                  aws_secret_access_key= AWS_SECRET_KEY)
s3_resource.Object(bucket,'output/dimRegion.csv').put(Body=csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': 'ZN7RJPJH8DMTHKQD',
  'HostId': 'pn/OAWi5B/0b4RUVNb80FZeI6EgQTzX1/OdPSUfIwiDemUZbk+whzPZqxNobdPDb4BLIl44nEG4fpQMtHaGZBA==',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'pn/OAWi5B/0b4RUVNb80FZeI6EgQTzX1/OdPSUfIwiDemUZbk+whzPZqxNobdPDb4BLIl44nEG4fpQMtHaGZBA==',
   'x-amz-request-id': 'ZN7RJPJH8DMTHKQD',
   'date': 'Tue, 27 Feb 2024 00:02:44 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"6dbd76c170a232d0df8ab70802eed2fe"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 1},
 'ETag': '"6dbd76c170a232d0df8ab70802eed2fe"',
 'ServerSideEncryption': 'AES256'}

In [120]:
csv_buffer.getvalue()

',fips,province_state,country_region,latitude,longitude,county,state\r\n0,,Anhui,China,31.826,117.226,New York City,New York\r\n1,,Anhui,China,31.826,117.226,Unknown,Rhode Island\r\n2,,Anhui,China,31.826,117.226,New York City,New York\r\n3,,Anhui,China,31.826,117.226,Unknown,Rhode Island\r\n4,,Anhui,China,31.826,117.226,New York City,New York\r\n5,,Anhui,China,31.826,117.226,Unknown,Rhode Island\r\n6,,Anhui,China,31.826,117.226,New York City,New York\r\n7,,Anhui,China,31.826,117.226,Unknown,Rhode Island\r\n8,,Anhui,China,31.826,117.226,New York City,New York\r\n9,,Anhui,China,31.826,117.226,Unknown,Rhode Island\r\n10,,Anhui,China,31.826,117.226,New York City,New York\r\n11,,Anhui,China,31.826,117.226,Unknown,Rhode Island\r\n12,,Anhui,China,31.826,117.226,New York City,New York\r\n13,,Anhui,China,31.826,117.226,Unknown,Rhode Island\r\n14,,Anhui,China,31.826,117.226,New York City,New York\r\n15,,Anhui,China,31.826,117.226,Unknown,Rhode Island\r\n16,,Anhui,China,31.826,117.226,New York Ci

In [88]:
dimDatesql=pd.io.sql.get_schema(dimDate.reset_index(),'dimDate')
print(''.join(dimDatesql))

CREATE TABLE "dimDate" (
"index" INTEGER,
  "fips" INTEGER,
  "date" TIMESTAMP,
  "year" INTEGER,
  "month" INTEGER,
  "day_of_week" INTEGER
)


In [89]:
factCovidsql=pd.io.sql.get_schema(factCovid.reset_index(),'factCovid')
print(''.join(factCovidsql))

CREATE TABLE "factCovid" (
"index" INTEGER,
  "fips" REAL,
  "province_state" TEXT,
  "country_region" TEXT,
  "confirmed" REAL,
  "deaths" REAL,
  "recovered" REAL,
  "active" REAL,
  "date" INTEGER,
  "positive" REAL,
  "negative" REAL,
  "hospitalizedcurrently" REAL,
  "hospitalized" REAL,
  "hospitalizeddischarged" REAL
)


In [90]:
dimHospitalsql=pd.io.sql.get_schema(dimHospital.reset_index(),'dimHospital')
print(''.join(dimHospitalsql))

CREATE TABLE "dimHospital" (
"index" INTEGER,
  "fips" REAL,
  "state_name" TEXT,
  "latitude" REAL,
  "longtitude" REAL,
  "hq_address" TEXT,
  "hospital_type" TEXT,
  "hospital_name" TEXT,
  "hq_city" TEXT,
  "hq_state" TEXT
)


In [91]:
dimRegionsql=pd.io.sql.get_schema(dimRegion.reset_index(),'dimRegion')
print(''.join(dimRegionsql))

CREATE TABLE "dimRegion" (
"index" INTEGER,
  "fips" REAL,
  "province_state" TEXT,
  "country_region" TEXT,
  "latitude" REAL,
  "longitude" REAL,
  "county" TEXT,
  "state" TEXT
)


In [92]:
pip install redshift-connector


Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: redshift-connector in c:\users\sumer\anaconda3\lib\site-packages (2.1.0)



In [93]:
import redshift_connector

In [94]:
conn = redshift_connector.connect(
    host="sumer.992382590719.ap-south-1.redshift-serverless.amazonaws.com",
    database="dev",
    user="admin",
    password="Simran*123"
 )

In [95]:
conn.autocommit=True

In [96]:
cursor=redshift_connector.Cursor=conn.cursor()

In [97]:
cursor.execute("""
CREATE TABLE "dimDate" (
"index" INTEGER,
  "fips" INTEGER,
  "date" TIMESTAMP,
  "year" INTEGER,
  "month" INTEGER,
  "day_of_week" INTEGER
)

""")

In [109]:
cursor.execute("""
CREATE TABLE "factCovid" (
  "fips" REAL,
  "province_state" TEXT,
  "country_region" TEXT,
  "confirmed" REAL,
  "deaths" REAL,
  "recovered" REAL,
  "active" REAL,
  "date" INTEGER,
  "positive" REAL,
  "negative" REAL,
  "hospitalizedcurrently" REAL,
  "hospitalized" REAL,
  "hospitalizeddischarged" REAL
)

""")

In [99]:
cursor.execute("""
CREATE TABLE "dimHospital" (
"index" INTEGER,
  "fips" REAL,
  "state_name" TEXT,
  "latitude" REAL,
  "longtitude" REAL,
  "hq_address" TEXT,
  "hospital_type" TEXT,
  "hospital_name" TEXT,
  "hq_city" TEXT,
  "hq_state" TEXT
)

""")

In [126]:
cursor.execute("""
CREATE TABLE "dimRegion" (
  "fips" REAL,
  "province_state" TEXT,
  "country_region" TEXT,
  "latitude" REAL,
  "longitude" REAL,
  "county" TEXT,
  "state" TEXT
)

""")

In [101]:
cursor.execute("""
copy dimDate from 's3://output-sumer1/output/dimDate.csv'
credentials 'aws_iam_role=arn:aws:iam::992382590719:role/redshiftrole'
delimiter ','
region 'ap-south-1'
IGNOREHEADER 1
""")

In [112]:
cursor.execute("""
copy factCovid from 's3://output-sumer1/output/factCovid.csv'
credentials 'aws_iam_role=arn:aws:iam::992382590719:role/redshiftrole'
delimiter ','
region 'ap-south-1'
IGNOREHEADER 1
""")

In [113]:
cursor.execute("""
copy dimHospital from 's3://output-sumer1/output/dimHospital.csv'
credentials 'aws_iam_role=arn:aws:iam::992382590719:role/redshiftrole'
delimiter ','
region 'ap-south-1'
IGNOREHEADER 1
""")

In [127]:
cursor.execute("""
copy dimRegion from 's3://output-sumer1/output/dimRegion.csv'
credentials 'aws_iam_role=arn:aws:iam::992382590719:role/redshiftrole'
delimiter ','
region 'ap-south-1'
IGNOREHEADER 1
""")